In [ ]:
# from quantile_rescale.py - to archive
# produce two corrected files - nc5 done, nc2 ____
    # to ../results/r72/quant_resc/ - check, mkdir ../results/r8/
# pilot n3_n4 chk (?) - done, itertools, solve nc3a, nc2b, ..., etc.
    # use Tybalt VAE as control (?) - 
# Use quant resc as pilot for mbatch across projects and data types:
    # what is fastest route to answer neg val question?

In [1]:
%whos

Interactive namespace is empty.


In [2]:
# Data handling
import pandas as pd
import glob
import itertools

In [3]:
def quantileRescale(src, dst):
    lower = 0.10
    upper = 0.90
    dstRange = dst.quantile([lower,upper])
    srcRange = src.quantile([lower,upper])
    scale = (srcRange.loc[upper] - srcRange.loc[lower]) / (dstRange.loc[upper] - dstRange.loc[lower])
    shift = (srcRange.loc[lower] / scale) - dstRange.loc[lower]
    scaleDF = src / scale - shift
    return scaleDF

In [ ]:
# Quantile - native Pandas function

In [5]:
# phase = 'Input' (to Quantile rescaling here)
inp_pths = sorted(
    glob.glob('../data/r72/*.tsv'), reverse = True)
inp_pths

['../data/r72/nc5_transcriptomics_cell-line+CPTAC.input-data.tsv',
 '../data/r72/nc2_transcriptomics_HCMI+CPTAC.input-data.tsv']

### Cycle point

In [6]:
# Name vars to clssfr_eval.ipynb
path_index_toggle = 0
cancer_count = 5
sys_togg = 'cell-line'
sys_file_names = 'cell-line+CPTAC'
sample_count_lbl = 'Cell line'
mdl_sys_inline = 'cell line'

In [35]:
path_index_toggle = 1
cancer_count = 2
sys_togg = 'hcmi'
sys_file_names = 'HMCI+CPTAC'
sample_count_lbl = 'HCMI'
mdl_sys_inline = 'HCMI'

In [36]:
inp_pths[path_index_toggle]

'../data/r72/nc2_transcriptomics_HCMI+CPTAC.input-data.tsv'

In [37]:
# Input load, check read the raw data for q rsc
input_df = pd.read_csv(
                inp_pths[path_index_toggle], # nc5 or nc2 file
                sep = '\t',
                index_col = 0)
out_name = inp_pths[path_index_toggle].split('/')[-1].split('.')[0]
print('inpt len:', len(input_df))
print('out_name:', out_name)

inpt len: 332
out_name: nc2_transcriptomics_HCMI+CPTAC


In [38]:
input_df.System.unique() # Automation for HCMI:
                            # sort order reverse alph
                            # assign to src_ref
                            # Use alt method to relying on sort order

array(['cptac', 'hcmi'], dtype=object)

In [60]:
input_df.System.value_counts()

System
cptac    245
hcmi      87
Name: count, dtype: int64

In [61]:
input_df.Cancer_type.value_counts()

Cancer_type
pancreatic-nos-ductal-ad    181
colon-nos-adeno             151
Name: count, dtype: int64

In [39]:
input_df.iloc[:, 2:].isna().sum().sum()

0

In [ ]:
input_df

##### Indirect check for zeroed out column

In [41]:
q_tst = input_df.iloc[:, 2:].quantile([.1, .9])

In [42]:
q_tst.loc[.9, :].min()

0.0

In [43]:
input_df = input_df.loc[:, (input_df != 0).any(axis=0)]

In [ ]:
input_df

##### Split by system - quant scale

In [20]:
# Cell line and CPTAC - trans and prot
    # src, dst to r72 to r8 conversion
src_lab = input_df[input_df.System == 'cell-line'] #
src = src_lab.iloc[:, 2:]
dst_lab = input_df[input_df.System == 'cptac'] #
dst = dst_lab.iloc[:, 2:]

In [47]:
# HCMI and CPTAC - trans and prot
src_lab = input_df[input_df.System == 'hcmi']
src = src_lab.iloc[:, 2:]
dst_lab = input_df[input_df.System == 'cptac']
dst = dst_lab.iloc[:, 2:]

In [ ]:
# BeatAML - trans (ngtv vals) and prot
 = input_df[input_df.System == 'cptac']

 = input_df[input_df.System == 'hcmi']


##### Quant Resc main

In [ ]:
# MBatch alignment
    # * dst (quant_resc) = ref (mbatch) <----

In [48]:
src_lab.iloc[:3, :3]

,System,Cancer_type,entrz_tran_1
improve_sample_id,,,
smpl_id_3018,hcmi,colon-nos-adeno,0.0433
smpl_id_3024,hcmi,colon-nos-adeno,0.5619
smpl_id_3036,hcmi,pancreatic-nos-ductal-ad,0.0898


In [49]:
dst_lab.iloc[:3, :3]

,System,Cancer_type,entrz_tran_1
improve_sample_id,,,
smpl_id_1996,cptac,colon-nos-adeno,0.0
smpl_id_1997,cptac,colon-nos-adeno,0.0
smpl_id_1998,cptac,colon-nos-adeno,0.0


In [ ]:
src # cell line or HCMI

In [ ]:
dst # CPTAC or BeatAML

#### Rescale

In [50]:
isect = src.columns.intersection(dst.columns)
scaled = quantileRescale(src[isect], dst[isect])

##### Confirm order preservation, to index merge

In [ ]:
src_lab

In [ ]:
src # Observation: order preserved for label re-install

In [ ]:
scaled # Observation: n = 261, cell line data scaled into human space

##### Continue

In [ ]:
# Target out

In [52]:
ls ../results/r72/quant-resc/

nc2_transcriptomics_HCMI+CPTAC.quant_resc.tsv
nc5_transcriptomics_cell-line+CPTAC.quant_resc.tsv


In [51]:
scaled = pd.concat(
    [src_lab[['System', 'Cancer_type']], scaled], axis = 1)

In [ ]:
scaled

In [53]:
quant_resc = pd.concat([scaled, dst_lab], axis  = 0)

In [ ]:
quant_resc

In [28]:
pwd

'/Users/karlberb/Documents/work/2024/bnch_mrk/mdl-sys-bnchmrk/code'

In [31]:
ls ../results/r72/quant-resc/

nc2_transcriptomics_HCMI+CPTAC.quant_resc.tsv
nc5_transcriptomics_cell-line+CPTAC.quant_resc.tsv


In [55]:
quant_resc.to_csv(
    '../results/r72/quant-resc/'+out_name+'.quant_resc.tsv',
    sep='\t')

##### read check

In [ ]:
rc = pd.read_csv(
    '../results/r72/quant-resc/'+out_name+'.quant_resc.tsv',
    sep='\t')
rc

In [58]:
rc.System.value_counts()

System
cptac    245
hcmi      87
Name: count, dtype: int64

In [59]:
rc.Cancer_type.value_counts()

Cancer_type
pancreatic-nos-ductal-ad    181
colon-nos-adeno             151
Name: count, dtype: int64

In [ ]:
ls ../results/r72/quant_resc/